In [ ]:
import piplite
await piplite.install("ipywidgets")

import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from pyodide.http import open_url

In [ ]:
base_url = "http://development.earthmaps.io/"
latitude = 65.1
longitude = -146.6

In [ ]:
# function to get SNAP freezing index data from the API
def get_freezing_index(lat, lon):
    fi_url = f"{base_url}degree_days/freezing_index/{lat}/{lon}?format=csv"
    fi_df = url_contents_to_df(fi_url, 2)
    return fi_df

In [ ]:
# load the CSV data
df = get_freezing_index(latitude, longitude)

In [ ]:
# function to compute design_index
def compute_and_display_design_index(start_year, end_year, n_coldest_years):
    
    df_slice = df[(df["year"] >= start_year) & (df["year"] < end_year)]
    design_index = round(df_slice.sort_values("value", ascending=False).iloc[0:n_coldest_years]["value"].mean())
    return design_index

In [ ]:
# create interactive widgets for input
start_year_slider = widgets.IntSlider(
    value=2030, min=1980, max=2099, step=1, description="Start Year:"
)
end_year_slider = widgets.IntSlider(
    value=2060, min=1981, max=2100, step=1, description="End Year:"
)
n_coldest_years_slider = widgets.IntSlider(
    value=3, min=1, max=10, step=1, description="Number of Coldest Years:",
    style={"description_width": "initial"}
)

# create an output widget for dynamic displaying of result
output = widgets.Output()

# function to handle widget changes
def handle_slider_change(change):
    with output:
        output.clear_output()

        if (
            end_year_slider.value - start_year_slider.value
            < n_coldest_years_slider.value
        ):
            start_year_slider.value = (
                end_year_slider.value - n_coldest_years_slider.value
            )
            output.clear_output()
        result = compute_and_display_design_index(
            start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
        )
        print(f"Design Freezing Index: {result} °F-days")

# attach the change handler to the sliders
start_year_slider.observe(handle_slider_change, names="value")
end_year_slider.observe(handle_slider_change, names="value")
n_coldest_years_slider.observe(handle_slider_change, names="value")

handle_slider_change(None)

# display the widgets with the title and initial result
display(start_year_slider, end_year_slider, n_coldest_years_slider, output)
initial_result = compute_and_display_design_index(
    start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
)
